<a href="https://colab.research.google.com/github/gonemyway/datn/blob/main/datn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/googledrive')

Mounted at /content/googledrive


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import random
import re

In [4]:
# Đọc dữ liệu từ file CSV
file_path = '/content/googledrive/MyDrive/chot1.csv'
data = pd.read_csv(file_path)
data

,Freq [GHz],"dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='12mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='1mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='12mm' l22='1mm'","dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='8.33333333333333mm' l12='1mm' l21='1mm' l22='1mm'",...,"dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='4.66666666666667mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='4.66666666666667mm' l12='12mm' l21='12mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='1mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='12mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='1mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='8.33333333333333mm' l22='12mm'","dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='12mm' l22='12mm'"
0,2.00,-0.212910,-0.223017,-0.243221,-0.277751,-0.238104,-0.244566,-0.261524,-0.293794,-0.281863,...,-0.564177,-0.579111,-0.632022,-0.629144,-0.635149,-0.647140,-0.769072,-0.737993,-0.733482,-0.742975
1,2.01,-0.213998,-0.224521,-0.245530,-0.281456,-0.239942,-0.246858,-0.264384,-0.297603,-0.285110,...,-0.578958,-0.594877,-0.652261,-0.646806,-0.653228,-0.665848,-0.794553,-0.760544,-0.756080,-0.766783
2,2.02,-0.215087,-0.226024,-0.247839,-0.285162,-0.241781,-0.249151,-0.267244,-0.301412,-0.288357,...,-0.593740,-0.610642,-0.672501,-0.664468,-0.671306,-0.684556,-0.820034,-0.783096,-0.778677,-0.790592
3,2.03,-0.216232,-0.227586,-0.250242,-0.289004,-0.243732,-0.251523,-0.270213,-0.305425,-0.291830,...,-0.609771,-0.627759,-0.694199,-0.683743,-0.691024,-0.704926,-0.847977,-0.808096,-0.803593,-0.816598
4,2.04,-0.217379,-0.229148,-0.252647,-0.292849,-0.245684,-0.253895,-0.273183,-0.309441,-0.295305,...,-0.625816,-0.644890,-0.715911,-0.703034,-0.710758,-0.725314,-0.875945,-0.833121,-0.828532,-0.842627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,5.96,-2.045830,-2.066182,-1.269520,-2.124684,-1.653563,-1.884780,-1.580972,-1.550803,-1.479099,...,-0.994571,-0.761300,-0.810719,-1.389341,-0.770665,-0.761417,-0.816818,-2.729492,-0.791306,-0.762244
397,5.97,-2.035801,-2.063912,-1.266476,-2.119232,-1.655492,-1.875854,-1.571367,-1.546874,-1.486832,...,-0.968431,-0.766122,-0.813996,-1.416761,-0.772421,-0.763178,-0.818771,-2.831186,-0.792010,-0.760271
398,5.98,-2.025772,-2.061643,-1.263432,-2.113764,-1.657424,-1.866918,-1.561763,-1.542942,-1.494568,...,-0.942312,-0.770947,-0.817275,-1.444204,-0.774180,-0.764939,-0.820727,-2.932991,-0.792715,-0.758298
399,5.99,-2.015827,-2.059467,-1.260389,-2.106810,-1.659583,-1.857088,-1.552280,-1.538681,-1.502614,...,-0.918307,-0.776036,-0.820731,-1.473881,-0.776166,-0.766810,-0.822802,-3.045581,-0.793610,-0.756272


In [5]:
filtered_data = data[(data['Freq [GHz]'].between(2.4, 3)) | (data['Freq [GHz]'].between(5.1, 5.6))]

In [6]:
filtered_data = filtered_data.drop(columns=['Freq [GHz]'])

In [7]:
absolute_sum = filtered_data.abs().sum()
absolute_sum

dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='1mm' l22='1mm'                      174.275618
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='4.66666666666667mm' l22='1mm'       157.116432
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='8.33333333333333mm' l22='1mm'       129.453040
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='1mm' l12='1mm' l21='12mm' l22='1mm'                     172.295225
dB(S(1,1)) [] - h1='5mm' h2='0.5mm' l11='4.66666666666667mm' l12='1mm' l21='1mm' l22='1mm'       133.170316
                                                                                                    ...    
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='8.33333333333333mm' l12='12mm' l21='12mm' l22='12mm'    640.954117
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='1mm' l22='12mm'                   626.340218
dB(S(1,1)) [] - h1='8mm' h2='1.5mm' l11='12mm' l12='12mm' l21='4.66666666666667mm' l22='12mm'    651.278316
dB(S(1,1)) [] - h1='8mm' h2=

In [8]:
absolute_sum_df = absolute_sum.to_frame(name='Absolute Sum')
absolute_sum_df.reset_index(inplace=True)
absolute_sum_df.columns = ['Parameter', 'Absolute Sum']

In [118]:
def extract_parameters(column_name):
    pattern = r"h1='(\d+\.?\d*)mm' h2='(\d+\.?\d*)mm' l11='(\d+\.?\d*)mm' l12='(\d+\.?\d*)mm' l21='(\d+\.?\d*)mm' l22='(\d+\.?\d*)mm'"
    match = re.search(pattern, column_name)
    if match:
        return [float(match.group(i)) for i in range(1, 7)]
    return [None] * 6

In [119]:
parameters = absolute_sum_df['Parameter'].apply(extract_parameters)
parameters_df = pd.DataFrame(parameters.tolist(), columns=['h1', 'h2', 'l11', 'l12', 'l21', 'l22'])
parameters_df['Sum S11'] = absolute_sum_df['Absolute Sum']
parameters_df.head()

,h1,h2,l11,l12,l21,l22,Sum S11
0,5.0,0.5,1.000000,1.0,1.000000,1.0,174.275618
1,5.0,0.5,1.000000,1.0,4.666667,1.0,157.116432
2,5.0,0.5,1.000000,1.0,8.333333,1.0,129.453040
3,5.0,0.5,1.000000,1.0,12.000000,1.0,172.295225
4,5.0,0.5,4.666667,1.0,1.000000,1.0,133.170316


In [120]:
def create_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  # Output layer
    return model

In [121]:
# Định nghĩa hàm đánh giá (fitness function)
def fitness_function(individual):
    sample_df = pd.DataFrame([individual], columns=['h1', 'h2', 'l11', 'l12', 'l21', 'l22'])
    sample_scaled = scaler_X.transform(sample_df)
    predicted_scaled = model.predict(sample_scaled)
    predicted = scaler_y.inverse_transform(predicted_scaled).flatten()
    return predicted[0]

In [122]:
# Các bước của GA: khởi tạo, lựa chọn, lai ghép, đột biến
def initialize_population_from_data(data, size, bounds):
    population = []
    for i in range(size):
        individual = data.iloc[random.randint(0, len(data) - 1)][['h1', 'h2', 'l11', 'l12', 'l21', 'l22']].values.tolist()
        # Đảm bảo các giá trị nằm trong phạm vi bounds
        individual = [max(min(individual[j], bounds[j][1]), bounds[j][0]) for j in range(len(individual))]
        population.append(individual)
    return population

def selection(population, fitnesses, num_parents):
    selected = np.argsort(fitnesses)[-num_parents:]
    return [population[i] for i in selected]

def crossover(parents, offspring_size):
    offspring = []
    crossover_point = np.uint8(len(parents[0]) / 2)
    for k in range(offspring_size):
        parent1_idx = k % len(parents)
        parent2_idx = (k + 1) % len(parents)
        offspring.append(parents[parent1_idx][:crossover_point] + parents[parent2_idx][crossover_point:])
    return offspring

def mutation(offspring, mutation_rate, bounds):
    for idx in range(len(offspring)):
        if random.random() < mutation_rate:
            gene_idx = random.randint(0, len(bounds) - 1)
            random_value = random.uniform(bounds[gene_idx][0], bounds[gene_idx][1])
            offspring[idx][gene_idx] = random_value
    return offspring


In [123]:
# Xác định phạm vi giá trị cho các tham số từ dữ liệu
bounds = [
    (parameters_df['h1'].min(), parameters_df['h1'].max()),
    (parameters_df['h2'].min(), parameters_df['h2'].max()),
    (parameters_df['l11'].min(), parameters_df['l11'].max()),
    (parameters_df['l12'].min(), parameters_df['l12'].max()),
    (parameters_df['l21'].min(), parameters_df['l21'].max()),
    (parameters_df['l22'].min(), parameters_df['l22'].max())
]

In [124]:
# Khởi tạo mô hình ANN và các bộ chuẩn hóa
X = parameters_df[['h1', 'h2', 'l11', 'l12', 'l21', 'l22']]
y = parameters_df['Sum S11'].values.reshape(-1, 1)

In [125]:
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

In [126]:
input_dim = X_scaled.shape[1]
model = create_ann_model(input_dim)
optimizer = Adam(learning_rate=0.0002)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
model.fit(X_scaled, y_scaled, epochs=1000, batch_size=32, validation_split=0.2)

Epoch 1/1000
58/58 [==============================] - 1s 5ms/step - loss: 0.9444 - mae: 0.8126 - val_loss: 1.0483 - val_mae: 0.8486
Epoch 2/1000
58/58 [==============================] - 0s 2ms/step - loss: 0.7229 - mae: 0.6969 - val_loss: 0.8874 - val_mae: 0.7707
Epoch 3/1000
58/58 [==============================] - 0s 2ms/step - loss: 0.5551 - mae: 0.5924 - val_loss: 0.7630 - val_mae: 0.7023
Epoch 4/1000
58/58 [==============================] - 0s 2ms/step - loss: 0.4314 - mae: 0.5050 - val_loss: 0.6518 - val_mae: 0.6410
Epoch 5/1000
58/58 [==============================] - 0s 2ms/step - loss: 0.3511 - mae: 0.4509 - val_loss: 0.5666 - val_mae: 0.5898
Epoch 6/1000
58/58 [==============================] - 0s 2ms/step - loss: 0.3031 - mae: 0.4189 - val_loss: 0.5026 - val_mae: 0.5499
Epoch 7/1000
58/58 [==============================] - 0s 3ms/step - loss: 0.2725 - mae: 0.3967 - val_loss: 0.4488 - val_mae: 0.5154
Epoch 8/1000
58/58 [==============================] - 0s 4ms/step - loss: 0.

In [127]:
# Thông số GA
population_size = 100
num_generations = 200
num_parents_mating = 50
mutation_rate = 0.1

In [128]:
# Khởi tạo quần thể ban đầu từ dữ liệu
population = initialize_population_from_data(parameters_df, population_size, bounds)

# Vòng lặp các thế hệ
for generation in range(num_generations):
    fitnesses = [fitness_function(individual) for individual in population]
    parents = selection(population, fitnesses, num_parents_mating)
    offspring_crossover = crossover(parents, population_size - len(parents))
    offspring_mutation = mutation(offspring_crossover, mutation_rate, bounds)
    population = parents + offspring_mutation

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
1/1 [==============================] - 0s 37ms/step


In [129]:
# Tìm cá thể tốt nhất sau các thế hệ
best_individual = population[np.argmax(fitnesses)]
best_fitness = max(fitnesses)

print(f"Best individual: {best_individual}")
print(f"Predicted Sum S11: {best_fitness}")

Best individual: [8.0, 0.5, 6.273697280031426, 12.0, 1.6793026303073035, 5.958203711816211]
Predicted Sum S11: 1145.8236083984375
